# Caravan-ESP: preprocesar caudal
***

***Autor:** Jesús Casado Rodríguez*<br>
***Fecha:** 05-08-2023*<br>

**Introducción:**<br>
Para compilar los datos de Caravan son necesarias dos tablas:
* Una tabla con el ID, longitud, latitud, nombre y país de cada una de las estaciones. Las coordenadas deben de estar en el sistema WGS84 (EPSG:4326).
* Una tabla con la serie temporal de caudal diario de cada una de las estaciones.

En este _notebook_ se cargan las estaciones seleccionadas para incluir en el conjunto de datos Caravan-ESP, y sus series de caudal diario. Las series de caudal se transforma en caudal específico (caudal dividido por área de la cuenca vertiente, mm/d) y se exportan para su posterior utilización en la generación de Caravan-ESP.

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import xarray as xr
import geopandas as gpd
from tqdm.notebook import tqdm

## Configuración

In [2]:
# ruta donde se ubican los datos del Anurio de Aforos
path_in = '../data/anuario_aforos/'

# cuencas
catchments = 'all' #'cantabrico'

# inicio y fin del periodo de estudio
start = '1980-10-01'
end = '2020-09-30'

# tamaño mínimo y máximo de la cuenca
area_min = 100 # km²
area_max = None # km²

## Procesado

In [3]:
if isinstance(catchments, str):
    if catchments == 'all':
        catchments= [dir for dir in os.listdir(path_in) if os.path.isdir(os.path.join(path_in, dir))]
    else:
        catchments = [catchments]

In [4]:
for catchment in tqdm(catchments):

    try:
        # cargar estaciones
        stns = pd.read_csv(f'{path_in}{catchment}/estaciones_seleccion.csv', index_col='indroea')
        stns.index = stns.index.astype(str)
    except:
        print(f'{catchment} no tiene la tabla de estaciones seleccionadas')
        continue

    try:
        # cargar series de caudal
        dis = pd.read_parquet(f'{path_in}{catchment}/caudal.parquet', columns=stns.index)
        dis = dis.loc[start:end,:]
    except:
        print(f'{catchment} no tiene la tabla con las series diarias de caudal')
        continue

    # calcular caudal específico
    dis_sp = dis / stns.suprest * 3600 * 24 * 10e-3 # mm/d
    dis_sp.index.name = 'date'

    # unir al resto de cuencas
    if 'stations' not in locals():
        stations = stns
    else:
        stations = pd.concat((stations, stns), axis=0)
    if 'discharge_sp' not in locals():
        discharge_sp = dis_sp
    else:
        discharge_sp = pd.concat((discharge_sp, dis_sp), axis=1)

  0%|          | 0/10 [00:00<?, ?it/s]

GUADIANA no tiene la tabla de estaciones seleccionadas


## Exportar

In [8]:
if len(catchments) == 1:
    path_out = f'{path_in}/{catchments[0]}/'
    discharge_sp.to_csv(f'{path_out}/caravan_caudal_especifico.csv', float_format='%.3f')
elif len(catchments) > 1:
    path_out = path_in
    stations.to_csv(f'{path_out}/caravan_estaciones.csv', float_format='%.3f')
    discharge_sp.to_csv(f'{path_out}/caravan_caudal_especifico.csv', float_format='%.3f')